In [2]:
import xml.etree.ElementTree as ET 




parquetName = "appleWatchHRdf.parquet.gzip"


path = "/home/chowder/Documents/personalData/Apple/"
exportPath = "export/apple_health_export/"

xmlFileName = "export.xml"
tree = ET.parse(path + exportPath + xmlFileName)

root = tree.getroot() 


rType = "HKQuantityTypeIdentifierHeartRate"
manufaturerField = "Apple Inc."
modelField = "Watch"





In [3]:
print(len(root.findall("./")))

1183315


In [4]:
print(len(root.findall("./Record")))

1182288


In [5]:
print(root.findall("./Record")[2].attrib)

{'type': 'HKQuantityTypeIdentifierHeartRate', 'sourceName': 'Abhik’s Apple\xa0Watch', 'sourceVersion': '7.6', 'device': '<<HKDevice: 0x303053250>, name:Apple Watch, manufacturer:Apple Inc., model:Watch, hardware:Watch6,1, software:7.6>', 'unit': 'count/min', 'creationDate': '2021-07-19 19:55:07 -0700', 'startDate': '2021-07-19 19:55:06 -0700', 'endDate': '2021-07-19 19:55:06 -0700', 'value': '78'}


In [6]:
print(root.findall("./Record")[2].attrib["device"].split(","))

['<<HKDevice: 0x303053250>', ' name:Apple Watch', ' manufacturer:Apple Inc.', ' model:Watch', ' hardware:Watch6', '1', ' software:7.6>']


In [7]:
r = root.findall("./Record")[2]
dev = {x.split(":")[0] : x.split(":")[1] for x in r.attrib["device"].split(",") if len(x.split(":")) == 2}
print(dev)

{'<<HKDevice': ' 0x303053250>', ' name': 'Apple Watch', ' manufacturer': 'Apple Inc.', ' model': 'Watch', ' hardware': 'Watch6', ' software': '7.6>'}


In [8]:
import pandas as pd

numRecords = 0
listRecords = []

def getAppleWatchInfo(record):
    # check if the type of record is right
    if (record["type"] == "HKQuantityTypeIdentifierHeartRate"):

        # parse the device to a dictonary to make sure we have the right one
        # raw text of record["device"] should look like this 
        # '<<HKDevice: 0x999999999>, name:Apple Watch, manufacturer:' +
        # 'Apple Inc., model:Watch, hardware:Watch6,1, software:7.6>'
        device = {x.split(":")[0].strip() : x.split(":")[1] 
                  for x in record["device"].split(",")
                  if len(x.split(":")) == 2}

        if(device["manufacturer"] == "Apple Inc." and
           device["model"] == "Watch"):
           return device
    
    return None


# for every element tagged Record
for r in root.findall("./Record"):
    # the data is in the attributes 
    record = r.attrib
    device = getAppleWatchInfo(record)
    if device is not None:
        numRecords += 1
        if numRecords % 100_000 == 0: print(numRecords)
        
        row = [pd.to_datetime(record["startDate"]),
               device["hardware"], 
               device["software"][:-1], #removes a hanging '>' from the field
               pd.to_datetime(record["creationDate"]), 
               float(record["value"])]

        listRecords.append(row)


print(len(listRecords))
print(numRecords)

100000
200000
300000
341786
341786


In [9]:
print(listRecords[0:10])


[[Timestamp('2021-07-19 19:55:06-0700', tz='pytz.FixedOffset(-420)'), 'Watch6', '7.6', Timestamp('2021-07-19 19:55:07-0700', tz='pytz.FixedOffset(-420)'), 78.0], [Timestamp('2021-07-19 19:55:11-0700', tz='pytz.FixedOffset(-420)'), 'Watch6', '7.6', Timestamp('2021-07-19 19:55:12-0700', tz='pytz.FixedOffset(-420)'), 79.0], [Timestamp('2021-07-19 19:55:14-0700', tz='pytz.FixedOffset(-420)'), 'Watch6', '7.6', Timestamp('2021-07-19 19:55:17-0700', tz='pytz.FixedOffset(-420)'), 80.0], [Timestamp('2021-07-19 19:55:21-0700', tz='pytz.FixedOffset(-420)'), 'Watch6', '7.6', Timestamp('2021-07-19 19:55:22-0700', tz='pytz.FixedOffset(-420)'), 82.0], [Timestamp('2021-07-19 19:55:23-0700', tz='pytz.FixedOffset(-420)'), 'Watch6', '7.6', Timestamp('2021-07-19 19:55:27-0700', tz='pytz.FixedOffset(-420)'), 83.0], [Timestamp('2021-07-19 19:55:30-0700', tz='pytz.FixedOffset(-420)'), 'Watch6', '7.6', Timestamp('2021-07-19 19:55:32-0700', tz='pytz.FixedOffset(-420)'), 84.0], [Timestamp('2021-07-19 19:55:34-0

In [10]:
print(listRecords[-10:])

[[Timestamp('2024-07-17 05:39:02-0700', tz='pytz.FixedOffset(-420)'), 'Watch6', '10.4', Timestamp('2024-07-17 05:44:06-0700', tz='pytz.FixedOffset(-420)'), 52.0], [Timestamp('2024-07-17 05:41:14-0700', tz='pytz.FixedOffset(-420)'), 'Watch6', '10.4', Timestamp('2024-07-17 05:49:53-0700', tz='pytz.FixedOffset(-420)'), 52.0], [Timestamp('2024-07-17 05:45:42-0700', tz='pytz.FixedOffset(-420)'), 'Watch6', '10.4', Timestamp('2024-07-17 05:50:01-0700', tz='pytz.FixedOffset(-420)'), 52.0], [Timestamp('2024-07-17 05:50:31-0700', tz='pytz.FixedOffset(-420)'), 'Watch6', '10.4', Timestamp('2024-07-17 05:59:07-0700', tz='pytz.FixedOffset(-420)'), 53.0], [Timestamp('2024-07-17 05:54:54-0700', tz='pytz.FixedOffset(-420)'), 'Watch6', '10.4', Timestamp('2024-07-17 06:06:12-0700', tz='pytz.FixedOffset(-420)'), 55.0], [Timestamp('2024-07-17 06:04:35-0700', tz='pytz.FixedOffset(-420)'), 'Watch6', '10.4', Timestamp('2024-07-17 06:06:12-0700', tz='pytz.FixedOffset(-420)'), 54.0], [Timestamp('2024-07-17 06:0

In [11]:
# convert the dateTime Strings to actual dateTimes


In [12]:

ColumnNames = ["sampleDT", "hardware", "software", "creationDate", "value"]

appleWatchHRDf = pd.DataFrame(columns=ColumnNames, data=listRecords)

appleWatchHRDf = appleWatchHRDf.set_index("sampleDT")


In [13]:
appleWatchHRDf

,hardware,software,creationDate,value
sampleDT,,,,
2021-07-19 19:55:06-07:00,Watch6,7.6,2021-07-19 19:55:07-07:00,78.0
2021-07-19 19:55:11-07:00,Watch6,7.6,2021-07-19 19:55:12-07:00,79.0
2021-07-19 19:55:14-07:00,Watch6,7.6,2021-07-19 19:55:17-07:00,80.0
2021-07-19 19:55:21-07:00,Watch6,7.6,2021-07-19 19:55:22-07:00,82.0
2021-07-19 19:55:23-07:00,Watch6,7.6,2021-07-19 19:55:27-07:00,83.0
...,...,...,...,...
2024-07-17 06:04:35-07:00,Watch6,10.4,2024-07-17 06:06:12-07:00,54.0
2024-07-17 06:08:51-07:00,Watch6,10.4,2024-07-17 06:10:58-07:00,55.0
2024-07-17 06:10:52-07:00,Watch6,10.4,2024-07-17 06:24:44-07:00,54.0


In [14]:
appleWatchHRDf.dtypes

hardware                                        object
software                                        object
creationDate    datetime64[ns, pytz.FixedOffset(-420)]
value                                          float64
dtype: object

In [18]:
appleWatchHRDf.to_parquet(path + parquetName,
              compression='gzip') 